## создание сводных для кажого бренда и фасовки, нужно сделать вычисления в этой таблице и соединить их вместе

In [1]:
import pandas as pd

### Бренды

In [2]:
brands = [
    'BlackBurn',
    'Burn',
    'B3',
    'Banger',
    'PeterRalf',
    'Overdose'
]

### Планы на каждый бренд

In [3]:
BB_Plan = 50000
B_Plan = 2350
B3_Plan = 327
BNG_Plan = 3450
PR_Plan = 80
OVD_Plan = 7650

### Вычисление доли фасовок и вычисление плана

In [4]:
df = pd.read_excel(r"D:\Analysis Burn\Кирилл Кузема\Планирование июль 23_подготовка.xlsx", sheet_name='Исходные данные')

In [147]:
# создание для каждого бренда таблицы с долей граммовки
for i in brands:
    exec(f"{i} = df.loc[df['Тип'] == '{i}'].groupby( \
        by=['Вес (гр)'] \
    )['Кг'].sum().reset_index()" )

    exec(f"{i}['доля'] = {i}['Кг'] / {i}['Кг'].sum() ")

    
    if i == 'BlackBurn':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {BB_Plan}")

    if i == 'Burn':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {B_Plan}")

    if i == 'B3':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {B3_Plan}")

    if i == 'Banger':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {BNG_Plan}")
        
    if i == 'PeterRalf':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {PR_Plan}")

    if i == 'Overdose':
        exec(f"{i}['планируемый_объем'] = {i}['доля'] * {OVD_Plan}")


2350
327
3450
80
7650


### создание таблиц для каждого бренда каждой фасовки

In [9]:
brands_dict = {
    'BlackBurn' : BlackBurn, 
    'Burn' : Burn,
    'B3' : B3, 
    'Banger' : Banger,
    'PeterRalf': PeterRalf, 
    'Overdose': Overdose
}

In [12]:
dfs = {}
for brand in brands:
   data = brands_dict[brand]
   for idx, row in data.iterrows():
      need_row = int(row['Вес (гр)'])
      dfs[f"{brand}_{str(need_row)}"] = df.loc[(df['Тип'] == brand) & (df['Вес (гр)'] == need_row)]

### Создание таблицы с планированием

In [224]:
pivot_tables = {}

for data in dfs:

    dfs.get(data)['year'] = dfs.get(data).loc[:, 'Месяц'].dt.year
    dfs.get(data)['month'] = dfs.get(data).loc[:, 'Месяц'].dt.month

    pivot_tables[data] = dfs.get(data).pivot_table(
        values=['Кг'],
        index=['Тип', 'Вес (гр)','Товар\вкус'],
        columns=['year', 'month'], 
        aggfunc="sum"
    ).round().reset_index()


    current_brand = pivot_tables[data]['Тип'][0]
    current_group = pivot_tables[data]['Вес (гр)'][0]

    for br in brands_dict:
        if br == current_brand:
            current_plan = brands_dict[br].loc[brands_dict[br]['Вес (гр)'] == current_group]['планируемый_объем'].iloc[0]
    

    # сделать двухэтажные заголовки обычными
    pivot_tables[data].columns = ['_'.join([str(x) for x in idx if x and x != 'Кг']) for idx in pivot_tables[data].columns]

    pivot_tables[data]['Последний месяц'] = pivot_tables[data].iloc[:,-1:]

    # добавить колонку Средний объем в месяц при наличии
    pivot_tables[data]['Средний объем в месяц при наличии'] = pivot_tables[data].median(axis=1, numeric_only=True)

    # добавить колонку Средний объем в месяц при наличии
    pivot_tables[data]['Наибольший объем в месяц'] = pivot_tables[data].max(axis=1, numeric_only=True)

    # добавить колонку Доля продаж исходя из ср-зн
    pivot_tables[data]['Доля продаж исходя из ср-зн'] = ( pivot_tables[data]['Средний объем в месяц при наличии'] / pivot_tables[data]['Средний объем в месяц при наличии'].sum() )

    # добавить колонку Планируемый объем
    pivot_tables[data]['Планируемый объем'] = current_plan * pivot_tables[data]['Доля продаж исходя из ср-зн']
    
    # добавить колонку Дополнительно
    pivot_tables[data]['Дополнительно'] = pivot_tables[data]['Планируемый объем'] * 1.1
    
    # добавить колонку Итоговый заказ на месяц
    pivot_tables[data]['Итоговый заказ на месяц'] = pivot_tables[data]['Дополнительно'] + pivot_tables[data]['Планируемый объем']

    # добавить колонку Итоговый заказ на следующий месяц
    pivot_tables[data]['Итоговый заказ на месяц вперед'] = pivot_tables[data]['Итоговый заказ на месяц'] * 1.1

    # добавить колонку Итоговый заказ на два месяца вперед
    pivot_tables[data]['Итоговый заказ на два месяца вперед'] = pivot_tables[data]['Итоговый заказ на месяц вперед'] * 1.1

    # добавить колонку Разница по отношению к прошлому месяцу 
    pivot_tables[data]['Разница по отношению к прошлому месяцу'] = pivot_tables[data]['Итоговый заказ на месяц'] - pivot_tables[data]['Последний месяц']


    

C:\Users\User\AppData\Local\Temp\ipykernel_10448\3775065021.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.get(data)['year'] = dfs.get(data).loc[:, 'Месяц'].dt.year
C:\Users\User\AppData\Local\Temp\ipykernel_10448\3775065021.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs.get(data)['month'] = dfs.get(data).loc[:, 'Месяц'].dt.month
C:\Users\User\AppData\Local\Temp\ipykernel_10448\3775065021.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [218]:
BlackBurn

,Вес (гр),Кг,доля,планируемый_объем
0,25,205489.34,0.320831,16041.531783
1,100,99551.30,0.155430,7771.475362
2,200,335451.00,0.523740,26186.992854


In [219]:
pivot_tables['BlackBurn_25'].to_excel(r"C:\Users\User\Desktop\test.xlsx")

In [225]:
pivot_tables['BlackBurn_25']

,Тип,Вес (гр),Товар\вкус,2022_1,2022_2,2022_3,2022_4,2022_5,2022_6,2022_7,...,Последний месяц,Средний объем в месяц при наличии,Наибольший объем в месяц,Доля продаж исходя из ср-зн,Планируемый объем,Дополнительно,Итоговый заказ на месяц,Итоговый заказ на месяц вперед,Итоговый заказ на два месяца вперед,Разница по отношению к прошлому месяцу
0,BlackBurn,25,After8 (Шоколад-Мята),72.0,36.0,83.0,42.0,48.0,38.0,73.0,...,70.0,50.0,83.0,0.003441,55.203317,60.723648,115.926965,127.519661,140.271628,45.926965
1,BlackBurn,25,Almond Icecream (Миндальное Мороженое),82.0,42.0,88.0,45.0,60.0,80.0,74.0,...,82.0,61.0,90.0,0.004198,67.348046,74.082851,141.430897,155.573987,171.131386,59.430897
2,BlackBurn,25,Ananas Shock (Кислый Ананас),437.0,146.0,405.0,182.0,268.0,294.0,335.0,...,388.0,294.0,437.0,0.020235,324.595502,357.055052,681.650554,749.815609,824.797170,293.650554
3,BlackBurn,25,Apple Shock (Кислое Зеленое Яблоко),184.0,118.0,126.0,2.0,151.0,200.0,207.0,...,244.0,172.0,255.0,0.011838,189.899409,208.889350,398.788759,438.667635,482.534399,154.788759
4,BlackBurn,25,Asian Lychee (Личи),61.0,35.0,87.0,36.0,33.0,62.0,67.0,...,77.0,54.0,87.0,0.003717,59.619582,65.581540,125.201122,137.721234,151.493358,48.201122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,BlackBurn,25,Tropic Jack (Джекфрут),NaN,NaN,NaN,NaN,NaN,NaN,744.0,...,114.0,93.0,744.0,0.006401,102.678169,112.945986,215.624155,237.186570,260.905227,101.624155
64,BlackBurn,25,Watermelon (Сладкий Арбуз),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,121.0,284.0,780.0,0.019546,313.554839,344.910322,658.465161,724.311677,796.742845,537.465161
65,BlackBurn,25,Ёlka (Ёлка),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,186.0,186.0,758.0,0.012802,205.356338,225.891972,431.248310,474.373141,521.810455,245.248310
66,BlackBurn,25,На Расслабоне (Банановое Суфле),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,701.0,701.0,1742.0,0.048247,773.950499,851.345549,1625.296049,1787.825653,1966.608219,924.296049
